## 1、字符串解析器 StrOutputParser

In [2]:
# 1、获取大模型
import os
import dotenv
from defusedxml.cElementTree import XMLParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY1')
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 2.调用大模型
response = llm.invoke("什么是爱情？")
print(type(response))

# 3.使用字符串解析器
parser = StrOutputParser()
parsed_response = parser.invoke(response)
print(type(parsed_response))
print(parsed_response)

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>
爱情是一种复杂而深刻的情感，通常涉及对另一个人的强烈依恋、关心和吸引。它可以表现为浪漫的情感，也可以是亲情、友情等多种形式的深厚情感。爱情不仅仅是生理上的吸引，还包括心理和情感上的连接。

在不同的文化和哲学中，爱情的定义和表现形式各不相同。它可以带来快乐和满足感，也可能伴随着痛苦和挑战。爱情常常涉及信任、理解、包容和支持，是人际关系中重要的一部分。

总的来说，爱情是一种让人感到充实和有意义的情感体验，能够激励人们追求更好的自己和更美好的生活。


## 2、JSON解析器 JsonOutputParser
实现方式
- 方式1：用户自己通过提示词指明返回Json格式
- 方式2：借助JsonOutputParser的 get_format_instructions() ，生成格式说明，指导模型输出JSON 结构

In [6]:
# 方式1：
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_prompt_template = ChatPromptTemplate.from_messages([
	("system", "你是一个靠谱的{role}"),
	("human", "{question}")
])

result = chat_model.invoke(
	chat_prompt_template.invoke(
		{"role": "人工智能专家",
		 "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式"
		 }
	)
)
print(result.content)
print(type(result))

# 获取一个JsonOutputParser的实例
json_parser = JsonOutputParser()
parsed_result = json_parser.invoke(result)
print(parsed_result)
print(type(parsed_result))

```json
{
  "q": "人工智能用英文怎么说？",
  "a": "Artificial Intelligence"
}
```
<class 'langchain_core.messages.ai.AIMessage'>
{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}
<class 'dict'>


In [7]:
# 举例2
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Return a JSON object.


In [8]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
joke_query = "告诉我一个笑话。"
# 定义Json解析器
parser = JsonOutputParser()
# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
	template="回答用户查询的问题\n满足格式为{format_instructions}\n问题是:{question}",
	partial_variables={
		"format_instructions": parser.get_format_instructions(),
	}
)
prompt = prompt_template.invoke(input={"question": joke_query})
# 调用大模型
response = chat_model.invoke(prompt)
#
json_result = parser.invoke(response)
print(json_result)
print(type(json_result))

{'joke': '为什么自行车不能站起来？因为它有两个轮子！'}
<class 'dict'>


知识拓展：

In [9]:
# 针对举例1
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_prompt_template = ChatPromptTemplate.from_messages([
	("system", "你是一个靠谱的{role}"),
	("human", "{question}")
])

# 获取一个JsonOutputParser的实例
json_parser = JsonOutputParser()

chain = chat_prompt_template | chat_model | json_parser
json_result = chain.invoke(
	{"role": "人工智能专家",
	 "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式"
	 }
)
print(json_result)
print(type(json_result))

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}
<class 'dict'>


In [11]:
# 针对举例2
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
joke_query = "告诉我一个脑经急转弯。"

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
	template="回答用户查询的问题\n满足格式为{format_instructions}\n问题是:{question}",
	partial_variables={
		"format_instructions": parser.get_format_instructions(),
	}
)
# 定义Json解析器
parser = JsonOutputParser()

chain = prompt_template | chat_model | parser
json_result = chain.invoke(input={"question": joke_query})
print(json_result)
print(type(json_result))

{'riddle': '什么东西越洗越脏？', 'answer': '水'}
<class 'dict'>


## 3、XML解析器 XMLOutputParser

注意：XMLOutputParser 不会直接将模型的输出保持为原始XML字符串，而是会解析XML并转换成Python字典 （或类似结构化的数据）。目的是为了方便程序后续处理数据，而不是单纯保留XML格式。

In [13]:
# 举例1：不使用XMLOutputParser，通过大模型的能力，返回xml格式数据
# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
# 测试模型的xml解析效果
actor_query = "生成周星驰的简短电影记录"
output = chat_model.invoke(f"""{actor_query}请将影片附在<movie></movie>标签中"""
                           )
print(type(output))  # <class 'langchain_core.messages.ai.AIMessage'>
print(output.content)

<class 'langchain_core.messages.ai.AIMessage'>
周星驰是一位著名的中国香港演员、导演、编剧和制片人，以其独特的喜剧风格和幽默感而闻名。他的电影作品通常充满了夸张的搞笑元素，结合了丰富的文化背景和社会讽刺。以下是一些他的经典电影记录：

<movie>
1. 《大话西游之月光宝盒》（1995）
   - 角色：至尊宝
   - 简介：讲述了至尊宝的爱情故事，以及他在时光穿梭中所经历的奇幻冒险。

2. 《唐伯虎点秋香》（1993）
   - 角色：唐伯虎
   - 简介：唐伯虎为追寻秋香假扮成书生，发生了一系列搞笑的爱情纠葛。

3. 《功夫》（2004）
   - 角色：无名
   - 简介：在1930年代的中国，一名小混混为成为功夫高手而拼搏的故事，融合了喜剧与动作元素。

4. 《食神》（1996）
   - 角色：周星驰
   - 简介：讲述了一位被打入低谷的厨师重归巅峰的励志故事，满是美食和搞笑。

5. 《西游降魔篇》（2013）
   - 角色：唐僧
   - 简介：对经典《西游记》的幽默改编，结合了爱情、奋斗与友情的主题。
</movie>

这些影片不仅展示了周星驰的表演才华，还体现了他对社会现象的深刻观察与幽默批判，成为华语电影经典。


In [14]:
from langchain_core.output_parsers import XMLOutputParser

# 举例2
output_parser = XMLOutputParser()
parsed_output = output_parser.get_format_instructions()
print(parsed_output)

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


In [16]:
# 使用parser.get_format_instructions()结构实现
from langchain_core.output_parsers import XMLOutputParser
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY1')
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
# 初始化语言大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 测试模型的xml解析效果
actor_query = "生成苍井空的简短电影记录"

# 定义XML解析器
parser = XMLOutputParser()

#生成提示词模板
prompt_template1 = PromptTemplate.from_template(
	template="用户的问题{question}\n满足格式为{format_instructions}\n",
)
prompt_template2 = prompt_template1.partial(format_instructions=parser.get_format_instructions())
response = llm.invoke(
	prompt_template2.invoke(
		input={"question": actor_query}
	)
)
print(response.content)

```xml
<filmography>
    <actress>
        <name>苍井空</name>
        <movies>
            <movie>
                <title>爱丽丝的幻想</title>
                <year>2004</year>
                <genre>剧情</genre>
            </movie>
            <movie>
                <title>午夜迷情</title>
                <year>2005</year>
                <genre>爱情</genre>
            </movie>
            <movie>
                <title>激情之夜</title>
                <year>2006</year>
                <genre>爱情</genre>
            </movie>
            <movie>
                <title>梦幻俏佳人</title>
                <year>2007</year>
                <genre>喜剧</genre>
            </movie>
        </movies>
    </actress>
</filmography>
```


In [17]:
xml_result = parser.invoke(response)
print(type(xml_result))
print(xml_result)
# 注意：XMLOutputParser 不会直接将模型的输出保持为原始XML字符串，而是会解析XML并转换成Python字典 （或类似结构化的数据）。目的是为了方便程序后续处理数据，而不是单纯保留XML格式。

<class 'dict'>
{'filmography': [{'actress': [{'name': '苍井空'}, {'movies': [{'movie': [{'title': '爱丽丝的幻想'}, {'year': '2004'}, {'genre': '剧情'}]}, {'movie': [{'title': '午夜迷情'}, {'year': '2005'}, {'genre': '爱情'}]}, {'movie': [{'title': '激情之夜'}, {'year': '2006'}, {'genre': '爱情'}]}, {'movie': [{'title': '梦幻俏佳人'}, {'year': '2007'}, {'genre': '喜剧'}]}]}]}]}


## 4、列表解析器 CommaSeparatedListOutputParser

In [18]:
# 举例1
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)
messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


In [19]:
# 举例2
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
# 创建解析器
output_parser = CommaSeparatedListOutputParser()
# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
	"生成5个关于{text}的列表.\n\n{format_instructions}",
	partial_variables={
		"format_instructions": output_parser.get_format_instructions()
	})
# 提示模板与输出解析器传递输出
# chat_prompt =
chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})
print(res)
print(type(res))

['科幻电影', '动作电影', '爱情电影', '恐怖电影', '喜剧电影']
<class 'list'>


## 5、日期解析器 DatetimeOutputParser (了解)

In [20]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z

Return ONLY this string, no other words!


In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
import os
import dotenv

# 加载环境
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY1')
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
# 初始化语言大模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_prompt = ChatPromptTemplate.from_messages([
	("system", "{format_instructions}"),
	("human", "{request}")
])
output_parser = DatetimeOutputParser()
chain = chat_prompt | chat_model | output_parser
response = chain.invoke({
	"request": "请告诉我2024年奥运会的开幕日期？",
	"format_instructions": output_parser.get_format_instructions()
})
print(response)
print(type(response))

2024-07-26 00:00:00
<class 'datetime.datetime'>
